<a href="https://colab.research.google.com/github/SandipNarbat/Crome-Homepage/blob/main/YouTube_Video_Transcripter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download YouTube Video's Audio

In [32]:
! pip install pytube -q 

In [33]:
from pytube import YouTube

In [34]:
VIDEO_URL = "https://www.youtube.com/watch?v=aHUjPQcBJN0" #obama

In [35]:
%ls



0.wav  2.wav  4.wav  6.wav  8.wav  sample_data/  ytaudio.wav
1.wav  3.wav  5.wav  7.wav  9.wav  ytaudio.mp4


In [36]:
#VIDEO_URL = 'https://www.youtube.com/watch?v=h-JVjs9AAmQ' #batman

In [37]:
#VIDEO_URL = 'https://youtu.be/qNJRGHk7sN8'

In [38]:
yt = YouTube(VIDEO_URL)

In [39]:
yt.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'ytaudio.mp4')

'/content/ytaudio.mp4'

In [40]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

# English ASR with HuggingSound

In [41]:
!pip install huggingsound -q 

In [42]:
from huggingsound import SpeechRecognitionModel


In [43]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [44]:
device

'cuda'

In [45]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)


INFO:huggingsound.speech_recognition.model:Loading model...


OUT OF MEMORY (OOM) error

# Audio Chunking

In [46]:
import librosa

In [47]:
input_file = '/content/ytaudio.wav'

In [48]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [49]:
import soundfile as sf


In [50]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [51]:
i

9

# Audio Transcription / ASR / Speech to Text

In [52]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav') 

In [53]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav',
 '/content/9.wav']

In [54]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


In [55]:
full_transcript = ' '

In [56]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [57]:
len(full_transcript)

1718

# Text Summarization 

In [58]:
from transformers import pipeline

In [59]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [60]:
summarized_text = summarization(full_transcript)

In [61]:
summarized_text[0]['summary_text']

' It was quite uunexpected journey with everything that happenvrymaking of the film even as thevents continued oso it a wonderful thing to be other to to watch batmani now that my partner and crime who want tod betaweare a rogue tonight because he thought that would be more appropriate for oididnot do .'

Text Chunking before Summarization

In [62]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
  print("Summarized text\n"+out)
  summarized_text.append(out)

  #print(summarized_text)

input text 
 we were taking the film arond thepress tour and despite my being lfacinated an boosterd igoved goodmuses that i'm healthy i'm good but i not with you and it's a great heartbrake for me because this movie means so much to was a fte your journey for me mand ireally sidit resharing ith y tonightit was quite uunexpected journey with everything that happenvrymaking of the film even as thevents continued oso it a wonderful thing to be other to to watch batmani now that my partner and crime who want tod betaweare a rogue tonight because he thought that would be more appropriate for oididnot do hawould k the incredible people hat onner brothers for allowing omkeis film ujason yarpanternov tobiamric mesion peoplic tc walter amada ashocomo n yukay er incredible in any case ialso really one effects on the bottom of my heartthe incredible crewthat we made this movie they are such incredible craftsman artist honestly dilandi who say personal questions the best prew that we ever worked 